## Data gathering and cleanup

### Intro

Looking for a project idea, I went through different examples of fancy data visualization. But those examples was mostly based on data, unavailable for Ukraine, whereas I was highly interested in a project, related to the local community. Therefore I decided to look in open data portals.

First I took a look at the main portal of Ukrainian open data (http://data.gov.ua/). Now it mostly consists of non-table documents without any search filter for a document type. There why I decided to look for other sources.

Secondly, I visit Lviv open data portal (http://opendata.city-adm.lviv.ua/). Here I found datasets with the history of thefts, robberies and frauds cases detected in Lviv during 2015. And it looked like a good idea to visualize a map of this events to detect most dangerous places in Lviv. Dataset contains over 13 thousands of crime cases, which seems to be enough to analyse a criminal picture of the city with population of 800 thousands people.

### Data preparation

Data was saved in three datasets with common structure.

In [17]:
import pandas as pd
theft = pd.read_csv("theft.csv", sep=";")
robbery = pd.read_csv("robbery.csv", sep=";")
fraud = pd.read_csv("fraud.csv", sep=";")

In [18]:
theft.head()

,Орган/район,Дата/час скоєння,Вулиця,Будинок
0,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,13.09.2015 17:05,22 січня (Рудно),11А
1,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,23.12.2015 10:02,Авіаційна,7
2,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,05.12.2015 13:20,Авіаційна,NaN
3,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,30.11.2015 19:30,Авіаційна,1
4,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,27.11.2015 11:00,Авіаційна,NaN


In [19]:
robbery.head()

,Орган/район,Дата/час скоєння,Вулиця,Будинок,EO особа
0,ФРАНКІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ...,12.04.2015 22:17,Айвазовського І.,10,ЗАЯВНИК
1,ГАЛИЦЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБЛ...,08.07.2015 10:52,Архипенка О.,NaN,УЧАСНИК ПОДІІ
2,ГАЛИЦЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБЛ...,30.03.2015 23:05,Лепкого Б.,NaN,ЗАЯВНИК
3,ЛИЧАКІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ...,27.08.2015 17:08,Хмельницького (Винники),176,ЗАЯВНИК
4,ШЕВЧЕНКІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІ...,27.10.2015 12:27,Базарна,50,ЗАЯВНИК


In [20]:
fraud.head()

,Орган,Дата скоєння,Вулиця,Будинок,EO особа
0,СИХІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБ...,31.03.2015 18:00,Антоненка-Давидовича Б.,NaN,ЗАЯВНИК
1,СИХІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБ...,25.04.2015 11:25,Антоненка-Давидовича Б.,4,ЗАЯВНИК
2,СИХІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБ...,14.08.2015 13:00,Антоненка-Давидовича Б.,NaN,ЗАЯВНИК
3,СИХІВСЬКИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ ОБ...,23.10.2015 17:06,Антоненка-Давидовича Б.,NaN,ЗАЯВНИК
4,ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...,26.03.2015 19:55,Авіаційна,3В,ЗАЯВНИК


Without a column "EO особа", which seems to be uninformable, all datasets have a similar structure. Though they could be united in one after few column names fixed.

In [21]:
del robbery["EO особа"]
del fraud["EO особа"]

robbery["Орган"] = robbery["Орган/район"]
del robbery["Орган/район"]

theft["Орган"] = theft["Орган/район"]
del theft["Орган/район"]

theft["Дата/час"] = theft["Дата/час скоєння"]
del theft["Дата/час скоєння"]

robbery["Дата/час"] = robbery["Дата/час скоєння"]
del robbery["Дата/час скоєння"]

fraud["Дата/час"] = fraud["Дата скоєння"]
del fraud["Дата скоєння"]

theft["Тип"] = "Крадіжка"
robbery["Тип"] = "Пограбування"
fraud["Тип"] = "Шахрайство"

In [16]:
criminal = pd.concat([theft, robbery, fraud])

Address in data consists of street name and house number. To project this data on the Lviv's map, I need to know latitude and longitude for each address. I have used OpenStreetMap Geocoding API to do this. But before I have done some data cleaning.

In [173]:
def clean_street(street):
    comma = street.find(",")
    if(comma > -1):
        street = street[:comma]
    point = street.find(".")
    if(point > -1):
        street = street[:point-1]
        
    return street

criminal["Вулиця"] = criminal.apply(lambda row: clean_street(row["Вулиця"]), axis=1)

In [38]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import importlib
import geo
importlib.reload(geo)
from geo import request_gecoding_google, request_gecoding_osm

In [33]:
cache = {}

In [39]:
def get_from_cache_or_request(row, cache):
    key = row["Вулиця"] + ":" + str(row["Будинок"])
    if(key not in cache):
        result = request_gecoding_google(row["Вулиця"], row["Будинок"])
        if(result is not None):
            cache[key] = result
        else: return None
    
    return cache[key]
    
criminal.loc[criminal["Latitude"].isnull(),"Координати"] = criminal[criminal["Latitude"].isnull()].apply(lambda row: get_from_cache_or_request(row, cache), axis=1)

In [40]:
criminal.loc[criminal["Координати"].notnull(),"Latitude"] = criminal[criminal["Координати"].notnull()].apply(lambda row: row["Координати"][0] if (row["Координати"] is not None) else None, axis=1)
criminal.loc[criminal["Координати"].notnull(),"Longitude"] = criminal[criminal["Координати"].notnull()].apply(lambda row: row["Координати"][1] if (row["Координати"] is not None) else None, axis=1)
del criminal["Координати"]
criminal.to_csv("criminal.csv", index=False)

In [41]:
criminal[criminal["Latitude"].isnull()]

,Будинок,Вулиця,Дата/час,Орган,Тип,Час,Latitude,Longtitude,"Час, секунди"


In [30]:
import pandas as pd
import numpy as np

def tryFloat(s):
    try:
        return float(s)
    except:
        return None
    

criminal = pd.read_csv("criminal.csv")
criminal["Latitude"] = criminal.apply(lambda row: tryFloat(row["Latitude"]), axis = 1)
criminal.loc[criminal["Latitude"].isnull(), "Longitude"] = None

Будинок                                                         7
Вулиця                                                  Авіаційна
Дата/час                                         23.12.2015 10:02
Орган           ЗАЛІЗНИЧНИЙ ВІДДІЛ ПОЛІЦІЇ ГУНП У ЛЬВІВСЬКІЙ О...
Тип                                                      Крадіжка
Час                                                      10:02:00
Latitude                                                     None
Longtitude                                                   None
Час, секунди                                                36120
Name: 1, dtype: object

With data from OSM and Google Maps I collected coordinates for total majority of cases. Only 58 adresses was not found. I removed them from dataset, because they are useless on map visualization.

In [43]:
criminal = criminal[criminal["Lontitude"].notnull()]
criminal.to_csv("criminal.csv", index=False, encoding='utf8')
len(criminal)

13329

Also it would be insteresting to split this data by time of a day and analyse a time distribution of criminal cases. For that I extracted time information from Date/time column. ggplot do not allow to build scale on datetime values, though I converted time to number of seconds.

In [263]:
import datetime

def datetime_to_time(dt_str):
    dt_str = dt_str[:16]
    return datetime.datetime.time(datetime.datetime.strptime(dt_str, '%d.%m.%Y %H:%M'))

def time_to_float(time):
    return datetime.timedelta(hours=time.hour,minutes=time.minute,seconds=time.second).total_seconds()

criminal["Час"] = criminal.apply(lambda row: datetime_to_time(row["Дата/час"]), axis=1)
criminal["Час, секунди"] = criminal.apply(lambda row: time_to_float(row["Час"]), axis=1)

In [264]:
criminal.to_csv("criminal.csv", index=False)